# Notebook
## Sistem Rekomendasi Obat berdasarkan _Weighted Hybrid Approach_

Notebook ini menyajikan penerapan dan eksperimentasi _weighted hybrid approach_ pada sistem rekomendasi obat. _Weighted hybrid approach_ yang diusulkan merupakan kombinasi antara .... dan .... . Beberapa teknik pembobotan yang akan dimasukkan dalam eksperimentasi antara lain ...., ...., dan .....

## 1. Pustaka

Impor pustaka yang dibutuhkan.

In [2]:
pip install plotly

Note: you may need to restart the kernel to use updated packages.


In [3]:
# pustaka eksternal
import pandas as pd
import plotly.graph_objects as go

# pustaka custom
from analysis import *

ModuleNotFoundError: No module named 'analysis'

## 2. Persiapan Set Data

Pada bagian ini, dilakukan persiapan set data yang dibutuhkan dalam penelitian. Karena set data yang sesuai untuk secara langsung digunakan dalam penelitian ini belum tersedia saat penelitian ini dikerjakan, maka Peneliti melakukan pengumpulan data dari beberapa sumber dan menggabungkannya.

Set data `WebMD` dan `DailyMed` digabungkan dalam struktur data DataFrame.

In [ ]:
# berkas WebMD Drug Reviews oleh Rohan Harode dari Kaggle
path_wmd = 'D:\JUPYTER NOTEBOOK\TUGAS AKHIR\data\webmd.csv'

# berkas DailyMed Drug dari situs web DailyMed
# yang dikompilasi dengan menggunakan generate_dailymed.py
path_dm = 'D:\JUPYTER NOTEBOOK\TUGAS AKHIR\data\dailyMed.csv'

df_wmd = pd.read_csv(path_wmd)
df_dm = pd.read_csv(path_dm)

Atribut yang diperlukan sebagai pelengkap data pada WebMD, antara lain:
1. nama obat secara umum pada kolom `sub_name`
2. daftar bahan aktif pada kolom `list_activeIngredient`, dan 
3. daftar bahan inaktif pada kolom `list_activeIngredient`.

Oleh karena itu, perlu dilakukan seleksi kolom pada DataFrame `df_dm` sebagai berikut.

In [ ]:
df_dm = df_dm[["SubName", "ListActiveIngredient", "ListInactiveIngredient"]]

Gabungkan DataFrame DailyMed `df_dm` dan DataFrame MebMD `df_wmd` dengan menggunakan kolom `sub_name` di `df_dm` dan kolom `Drug` di `df_wmd` sebagai indeks pengabungan.

In [ ]:
df_drugs = pd.merge(df_dm, df_wmd, left_on = "SubName", right_on = "Drug")

## 3. Analisis Data

Dalam penelitian ini, analisis dilakukan pada `df_drugs` untuk memahami tipe data dan mengidentifikasi kesalahan. Informasi tersebut digunakan untuk menentukan prosedur yang sesuai dalam tahap prapemrosesan.

Kita mulai dengan menghitung jumlah masing-masing nilai yang berbeda pada kolom `Drug` untuk mengetahui jumlah baris data untuk setiap obat. Informasi ini diperlukan untuk memastikan kecukupan data penelitian.

In [ ]:
display(df_drugs["Drug"].value_counts())

Pada sel tersebut, kita dapat melihat rentang nilai dari setiap atribut. untuk attribut sub_name: 74 kategori, List_activeIngredient: 137 kategori, list_inactiveIngredient: 132 kategori, Age: 11 kategori, Condition: 405 kategori, Date: 3884 kategori, Drug:74 kategori, DrugId: 74 kategori, EaseofUse: 5 kategori, Effectiveness: 5 kategori, Reviews: 16967 kategori, Satisfaction: 5 kategori, Sex: 2 kategori, Sides: 69 kategori, UsefulCount: 94 kategori. 

## Type Data

In [ ]:
df_drugs.dtypes

Untuk memeriksa tipe data dari masing2 varibael/kolom/fitur dengan menggunakan .dtypes. untuk dtype (object) pada pandas digunakan untuk tipe data Teks atau campuran nilai numerik dan non-numeric. untuk dtype (int64) pada pandas digunakan untuk tipe data Integer numbers. Pada sel diatas, kita dapat melihat bahwa 2 jenis type data yang muncul yaitu object dan int64.

In [ ]:
for col in df_drugs:
    s = pd.get_dummies(df_drugs[col],prefix='', prefix_sep='').sum() 
    print(col, ":", s)

Pada sel tersebut, kita dapat melihat bahwa ada 74 nama obat dalam `df_drugs`. Obat dengan baris data terbanyak adalah `lisinopril` (12.807), sementara obat dengan baris data tersedikit adalah `cromolyn sodium`, `caffeine citrate`, dan `bexarotene` (2).

Jika menggunakan 1000 baris data sebagai ambang batas kecukupan data, maka persentase jumlah obat disajikan sebagai berikut.

In [ ]:
n_sufficient = sum(x >= 1000 for x in df_drugs["Drug"].value_counts())
n_insufficient = sum(x < 1000 for x in df_drugs["Drug"].value_counts())
fig = go.Figure(data=[go.Pie(labels=["Cukup", "Tidak Cukup"],
                             values=[n_sufficient, n_insufficient], hole = 0.5)])
fig.show()

Memperhatikan persentase tersebut, maka diperlukan penambahan lebih banyak baris data untuk setiap obat dengan total baris data yang kurang dari 500. Untuk saat ini, kita cukup gunakan baris data obat yang cukup.

In [ ]:
list_sufficient = [val for val, cnt in df_drugs["Drug"].value_counts().iteritems() if cnt >= 500]
df_drugs_sub = df_drugs[df_drugs["Drug"].isin(list_sufficient)]

Untuk menerapkan _item-based collaborative filtering_ pada sistem rekomendasi obat, dibutuhkan matriks pengguna/obat. Pada matriks tersebut terdapat _"user id"_, _"drug id"_, dan _"drug rating"_. Ilustrasi matriks tersebut adalah sebagai berikut.

|  | drug<sub>1</sub> | drug<sub>2</sub> | drug<sub>3</sub> |
| :---: | :---: | :---: | :---: |
| user<sub>1</sub> | rating<sub>user<sub>1</sub>,drug<sub>1</sub></sub> | rating<sub>user<sub>1</sub>,drug<sub>2</sub></sub> | rating<sub>user<sub>1</sub>,drug<sub>3</sub></sub> |
| user<sub>2</sub> | rating<sub>user<sub>2</sub>,drug<sub>1</sub></sub> | rating<sub>user<sub>2</sub>,drug<sub>2</sub></sub> | rating<sub>user<sub>2</sub>,drug<sub>3</sub></sub> |
| user<sub>3</sub> | rating<sub>user<sub>3</sub>,drug<sub>1</sub></sub> | rating<sub>user<sub>3</sub>,drug<sub>2</sub></sub> | rating<sub>user<sub>3</sub>,drug<sub>3</sub></sub> |
| user<sub>4</sub> | rating<sub>user<sub>4</sub>,drug<sub>1</sub></sub> | rating<sub>user<sub>4</sub>,drug<sub>2</sub></sub> | rating<sub>user<sub>4</sub>,drug<sub>3</sub></sub> |
| user<sub>5</sub> | rating<sub>user<sub>5</sub>,drug<sub>1</sub></sub> | rating<sub>user<sub>5</sub>,drug<sub>2</sub></sub> | rating<sub>user<sub>5</sub>,drug<sub>3</sub></sub> |


_"user id"_ tidak tersedia dalam `df_drugs`. Oleh karena itu, ....

_"drug id"_ tersedia dalam `df_drugs`. Oleh karena itu, ....

_"drug rating"_ tidak tersedia dalam `df_drugs`. Oleh karena itu, ....

In [ ]:
df_drugs_sub.pivot_table(columns=["Sex", "Age", "Condition"], aggfunc="size")

## Mengidentifikasi Kesalahan 

In [ ]:
df_drugs_sub.isnull().sum()

Dalam mengidentifikasi kesalahahan dapat dilakukan agregasi data  menggunakan fungsi sum(). Berdasarkan analisis diatas dapat dilihat bahwa dalam dataset tersebut semua kolom tidak memiliki nilai kosong atau NULL kecuali kolom Reviews dengan 16 missing value.

In [ ]:
data1 = df_drugs.drop(columns = ['Reviews'])
for col in data1:
    s = pd.get_dummies(df_drugs[col],prefix='', prefix_sep='').sum() 
    print(col, ":", s)

# s = pd.get_dummies(data1,prefix='', prefix_sep='').sum().sort()
# s


Berdasarkan cell code diatas, tidak ditemukan noise atau nilai aneh/ganjil untuk setiap field atau kolom. Namun terdapat nilai yang kosong seperti pada atribut 'Age'

## Membuat DataFrame

In [ ]:
df_drugs.columns

In [ ]:
df_drugs.rename(columns={"DrugId": "ItemId"}, inplace = True)
df_drugs.columns

In [ ]:
# Data Drug
df_item = pd.DataFrame({'ItemId':df_drugs['ItemId'],'Drug':df_drugs['Drug'], 'ListActiveIngredient':df_drugs['ListActiveIngredient'], 'ListInactiveIngredient':df_drugs['ListInactiveIngredient'],'Sides':df_drugs['Sides']})
df_item

In [ ]:
# Data User
df_user = pd.DataFrame({'Age':df_drugs['Age'], 'Sex':df_drugs['Sex'], 'Condition':df_drugs['Condition']})

In [ ]:
df_user['UserId'] = df_user.index
print(df_user)

In [ ]:
df_user = pd.DataFrame({'UserId':df_user['UserId'], 'Age':df_drugs['Age'], 'Sex':df_drugs['Sex'], 'Condition':df_drugs['Condition']})
df_user

In [ ]:
# Data Rating
df_rating = pd.DataFrame({'UserId':df_user['UserId'], 'ItemId':df_drugs['ItemId'], 'Effectiveness':df_drugs['Effectiveness'], 'EaseofUse':df_drugs['EaseofUse'],'Satisfaction':df_drugs['Satisfaction']})
df_rating

## 4. Prapemrosesan

...

## 5. Pembangunan Model Rekomendasi

...


## 6. Pengujian Model Rekomendasi

...

---